In [1]:
import os
import requests
import pandas as pd
from sklearn.feature_selection import SelectKBest,f_classif

In [2]:
import sqlite3

In [3]:
from tfx.components import CsvExampleGen

2024-03-04 03:26:02.249806: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-03-04 03:26:08.390519: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer.so.7'; dlerror: libnvinfer.so.7: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: /usr/local/cuda/lib64:/usr/local/cuda/lib:/usr/local/lib/x86_64-linux-gnu:/usr/local/nvidia/lib:/usr/local/nvidia/lib64:/usr/local/nvidia/lib:/usr/local/nvidia/lib64
2024-03-04 03:26:08.390847: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer_plugin.so.7'; dlerror: libnvinfer_plugin.so.7: cannot open shared object file: No such 

In [4]:
from tfx.components import StatisticsGen

In [5]:
from tfx.components import ExampleValidator

In [6]:
import tensorflow_data_validation as tfdv

In [7]:
from tfx.components import SchemaGen

In [8]:
from tensorflow_metadata.proto.v0 import schema_pb2

In [9]:
from tfx.dsl.components.common.importer import Importer

In [10]:
from tfx.types import standard_artifacts

In [11]:
from tfx.orchestration.experimental.interactive.interactive_context import InteractiveContext

In [12]:
from tfx.components import Transform

# Carga del dataset

In [13]:
## download the dataset
# Directory of the raw data files
_data_root = './data/covertype'
# Path to the raw training data
_data_filepath = os.path.join(_data_root, 'covertype_train.csv')
# Download data
os.makedirs(_data_root, exist_ok=True)
if not os.path.isfile(_data_filepath):
    #https://archive.ics.uci.edu/ml/machine-learning-databases/covtype/
    url = 'https://docs.google.com/uc?export= \
    download&confirm={{VALUE}}&id=1lVF1BCWLH4eXXV_YOJzjR7xZjj-wAGj9'
    r = requests.get(url, allow_redirects=True, stream=True)
    open(_data_filepath, 'wb').write(r.content)

In [14]:
df=pd.read_csv("data/covertype/covertype_train.csv")

In [15]:
df.head()

,Elevation,Aspect,Slope,Horizontal_Distance_To_Hydrology,Vertical_Distance_To_Hydrology,Horizontal_Distance_To_Roadways,Hillshade_9am,Hillshade_Noon,Hillshade_3pm,Horizontal_Distance_To_Fire_Points,Wilderness_Area,Soil_Type,Cover_Type
0,2991,119,7,67,11,1015,233,234,133,1570,Commanche,C7202,1
1,2876,3,18,485,71,2495,192,202,144,1557,Commanche,C7757,1
2,3171,315,2,277,9,4374,213,237,162,1052,Rawah,C7745,0
3,3087,342,13,190,31,4774,193,221,166,752,Rawah,C7745,0
4,2835,158,10,212,41,3596,231,242,141,3280,Rawah,C4744,1


In [16]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 116203 entries, 0 to 116202
Data columns (total 13 columns):
 #   Column                              Non-Null Count   Dtype 
---  ------                              --------------   ----- 
 0   Elevation                           116203 non-null  int64 
 1   Aspect                              116203 non-null  int64 
 2   Slope                               116203 non-null  int64 
 3   Horizontal_Distance_To_Hydrology    116203 non-null  int64 
 4   Vertical_Distance_To_Hydrology      116203 non-null  int64 
 5   Horizontal_Distance_To_Roadways     116203 non-null  int64 
 6   Hillshade_9am                       116203 non-null  int64 
 7   Hillshade_Noon                      116203 non-null  int64 
 8   Hillshade_3pm                       116203 non-null  int64 
 9   Horizontal_Distance_To_Fire_Points  116203 non-null  int64 
 10  Wilderness_Area                     116203 non-null  object
 11  Soil_Type                           116

# Selección de características
- El dataset a usar cuenta con 13 columnas, donde la columna objetivo es 'Cover_Type'.
- Dentro de los predictores hay 10 columnas numéricas y 2 categóricas
- En esta sección se realiza una selección de las características numéricas

In [17]:
# Separación de  las caracteristicas de entrada y de destino
X=df.drop("Cover_Type",axis=1)
y=df["Cover_Type"]

# Extracción de características numéricas
X_n=X.select_dtypes(include=['number'])

# Seleccción de las mejores caraterísticas con SelectKBest
selector=SelectKBest(f_classif, k=8)
X_new=selector.fit_transform(X_n, y)

## Características seleccionadas

In [18]:
pd.DataFrame({"Columnas":X_n.columns,"Retain":X_n.columns.isin(selector.get_feature_names_out())})

,Columnas,Retain
0,Elevation,True
1,Aspect,False
2,Slope,True
3,Horizontal_Distance_To_Hydrology,True
4,Vertical_Distance_To_Hydrology,True
5,Horizontal_Distance_To_Roadways,True
6,Hillshade_9am,True
7,Hillshade_Noon,True
8,Hillshade_3pm,False
9,Horizontal_Distance_To_Fire_Points,True


In [19]:
new_dataframe=pd.DataFrame(X_new,columns=selector.get_feature_names_out())

In [20]:
new_dataframe.head()

,Elevation,Slope,Horizontal_Distance_To_Hydrology,Vertical_Distance_To_Hydrology,Horizontal_Distance_To_Roadways,Hillshade_9am,Hillshade_Noon,Horizontal_Distance_To_Fire_Points
0,2991,7,67,11,1015,233,234,1570
1,2876,18,485,71,2495,192,202,1557
2,3171,2,277,9,4374,213,237,1052
3,3087,13,190,31,4774,193,221,752
4,2835,10,212,41,3596,231,242,3280


In [21]:
new_dataframe["Wilderness_Area"]=df["Wilderness_Area"]
new_dataframe["Soil_Type"]=df["Soil_Type"]
new_dataframe["Cover_Type"]=y

In [22]:
new_dataframe.head()

,Elevation,Slope,Horizontal_Distance_To_Hydrology,Vertical_Distance_To_Hydrology,Horizontal_Distance_To_Roadways,Hillshade_9am,Hillshade_Noon,Horizontal_Distance_To_Fire_Points,Wilderness_Area,Soil_Type,Cover_Type
0,2991,7,67,11,1015,233,234,1570,Commanche,C7202,1
1,2876,18,485,71,2495,192,202,1557,Commanche,C7757,1
2,3171,2,277,9,4374,213,237,1052,Rawah,C7745,0
3,3087,13,190,31,4774,193,221,752,Rawah,C7745,0
4,2835,10,212,41,3596,231,242,3280,Rawah,C4744,1


In [23]:
new_dataframe.to_csv('data/sel/data_selection.csv', index=False)

# Data Pipeline

## Configuración el contexto interactivo

In [24]:
metadata_path = 'meta/metadata.db'

context = InteractiveContext(pipeline_root="meta")

## Generación de ejemplos

In [25]:
example_gen = CsvExampleGen(input_base='data/sel')
context.run(example_gen)

ExecutionResult(
    component_id: CsvExampleGen
    execution_id: 1
    outputs:
        examples: OutputChannel(artifact_type=Examples, producer_component_id=CsvExampleGen, output_key=examples, additional_properties={}, additional_custom_properties={}))

## Generación de estadísticas

In [26]:
stats_gen= StatisticsGen(examples=example_gen.outputs['examples'],)
context.run(stats_gen)

ExecutionResult(
    component_id: StatisticsGen
    execution_id: 2
    outputs:
        statistics: OutputChannel(artifact_type=ExampleStatistics, producer_component_id=StatisticsGen, output_key=statistics, additional_properties={}, additional_custom_properties={}))

In [27]:
context.show(stats_gen.outputs['statistics'])

## Esquema

In [28]:
schema_gen = SchemaGen(statistics=stats_gen.outputs['statistics'])
context.run(schema_gen)

ExecutionResult(
    component_id: SchemaGen
    execution_id: 3
    outputs:
        schema: OutputChannel(artifact_type=Schema, producer_component_id=SchemaGen, output_key=schema, additional_properties={}, additional_custom_properties={}))

In [29]:
context.show(schema_gen.outputs['schema'])

,Type,Presence,Valency,Domain
Feature name,,,,
'Cover_Type',INT,required,,-
'Elevation',INT,required,,-
'Hillshade_9am',INT,required,,-
'Hillshade_Noon',INT,required,,-
'Horizontal_Distance_To_Fire_Points',INT,required,,-
'Horizontal_Distance_To_Hydrology',INT,required,,-
'Horizontal_Distance_To_Roadways',INT,required,,-
'Slope',INT,required,,-
'Soil_Type',STRING,required,,'Soil_Type'


,Values
Domain,
'Soil_Type',"'C2702', 'C2703', 'C2704', 'C2705', 'C2706', 'C2717', 'C3501', 'C3502', 'C4201', 'C4703', 'C4704', 'C4744', 'C4758', 'C5101', 'C6101', 'C6102', 'C6731', 'C7101', 'C7102', 'C7103', 'C7201', 'C7202', 'C7700', 'C7701', 'C7702', 'C7709', 'C7710', 'C7745', 'C7746', 'C7755', 'C7756', 'C7757', 'C7790', 'C8703', 'C8707', 'C8708', 'C8771', 'C8772', 'C8776', 'C5151'"
'Wilderness_Area',"'Cache', 'Commanche', 'Neota', 'Rawah'"


## Configuración del esquema

### Carga del esquema

In [30]:
schema_path = schema_gen.outputs['schema'].get()[0].uri + '/schema.pbtxt'
schema = tfdv.load_schema_text(schema_path)

### Modificación de dominios

In [31]:
tfdv.set_domain(schema, 'Hillshade_9am', schema_pb2.IntDomain(min=0, max=255))

In [32]:
tfdv.set_domain(schema, 'Hillshade_Noon', schema_pb2.IntDomain(min=0, max=255))

In [33]:
tfdv.set_domain(schema, 'Slope', schema_pb2.IntDomain(min=0, max=90))

In [34]:
tfdv.set_domain(schema, 'Cover_Type', schema_pb2.StringDomain(value=['0', '1', '2', '3', '4', '5', '6']))

### Actualización del esquema

In [35]:
tfdv.write_schema_text(schema, schema_path)

In [36]:
context.show(schema_gen.outputs['schema'])

,Type,Presence,Valency,Domain
Feature name,,,,
'Cover_Type',INT,required,,'Cover_Type_domain'
'Elevation',INT,required,,-
'Hillshade_9am',INT,required,,min: 0; max: 255
'Hillshade_Noon',INT,required,,min: 0; max: 255
'Horizontal_Distance_To_Fire_Points',INT,required,,-
'Horizontal_Distance_To_Hydrology',INT,required,,-
'Horizontal_Distance_To_Roadways',INT,required,,-
'Slope',INT,required,,min: 0; max: 90
'Soil_Type',STRING,required,,'Soil_Type'


,Values
Domain,
'Soil_Type',"'C2702', 'C2703', 'C2704', 'C2705', 'C2706', 'C2717', 'C3501', 'C3502', 'C4201', 'C4703', 'C4704', 'C4744', 'C4758', 'C5101', 'C6101', 'C6102', 'C6731', 'C7101', 'C7102', 'C7103', 'C7201', 'C7202', 'C7700', 'C7701', 'C7702', 'C7709', 'C7710', 'C7745', 'C7746', 'C7755', 'C7756', 'C7757', 'C7790', 'C8703', 'C8707', 'C8708', 'C8771', 'C8772', 'C8776', 'C5151'"
'Wilderness_Area',"'Cache', 'Commanche', 'Neota', 'Rawah'"
'Cover_Type_domain',"'0', '1', '2', '3', '4', '5', '6'"


# Entornos de esquema

### Creación de subconjunto de inferencia

In [37]:
new_dataframe.drop("Cover_Type",axis=1).to_csv('data/inference/inference.csv', index=False)

### Validación del subconjunto de inferencia con esquema actual
Como se puede apreciar, el esquema actual detecta anomalías en el conjunto de inferencias ya que no cuentan con las etiquetas

In [38]:
serving_anomalies = tfdv.validate_statistics(statistics=tfdv.generate_statistics_from_csv("data/inference/inference.csv"), schema=schema)

tfdv.display_anomalies(serving_anomalies)

Instructions for updating:
Use eager execution and: 
`tf.data.TFRecordDataset(path)`


Instructions for updating:
Use eager execution and: 
`tf.data.TFRecordDataset(path)`


,Anomaly short description,Anomaly long description
Feature name,,
'Cover_Type',Column dropped,Column is completely missing


### Añadir entornos de esquema para los conjuntos de entrenamiento e inferencia

In [39]:
schema.default_environment.append('TRAINING')
schema.default_environment.append('INFERENCIA')

### Especificar que la salida ("Cover_Type") no está en el entorno de inferencia

In [40]:
tfdv.get_feature(schema, 'Cover_Type').not_in_environment.append('INFERENCIA')

### Verificación de los entornos del esquema

#### Verificación de rangos de dominio

In [41]:
tfdv.display_schema(schema=schema)

,Type,Presence,Valency,Domain
Feature name,,,,
'Cover_Type',INT,required,,'Cover_Type_domain'
'Elevation',INT,required,,-
'Hillshade_9am',INT,required,,min: 0; max: 255
'Hillshade_Noon',INT,required,,min: 0; max: 255
'Horizontal_Distance_To_Fire_Points',INT,required,,-
'Horizontal_Distance_To_Hydrology',INT,required,,-
'Horizontal_Distance_To_Roadways',INT,required,,-
'Slope',INT,required,,min: 0; max: 90
'Soil_Type',STRING,required,,'Soil_Type'


,Values
Domain,
'Soil_Type',"'C2702', 'C2703', 'C2704', 'C2705', 'C2706', 'C2717', 'C3501', 'C3502', 'C4201', 'C4703', 'C4704', 'C4744', 'C4758', 'C5101', 'C6101', 'C6102', 'C6731', 'C7101', 'C7102', 'C7103', 'C7201', 'C7202', 'C7700', 'C7701', 'C7702', 'C7709', 'C7710', 'C7745', 'C7746', 'C7755', 'C7756', 'C7757', 'C7790', 'C8703', 'C8707', 'C8708', 'C8771', 'C8772', 'C8776', 'C5151'"
'Wilderness_Area',"'Cache', 'Commanche', 'Neota', 'Rawah'"
'Cover_Type_domain',"'0', '1', '2', '3', '4', '5', '6'"


#### Verificación de la existencia de entornos

In [42]:
# Entornos
schema.default_environment

['TRAINING', 'INFERENCIA']

#### Verificación de features no tenidos en cuenta en el entorno de inferencia

In [43]:
for feature in schema.feature:
    if feature.not_in_environment:
        print(feature.name)

Cover_Type


#### Validación del conjunto de inferencia con el nuevo entorno

In [44]:
serving_anomalies = tfdv.validate_statistics(statistics=tfdv.generate_statistics_from_csv("data/inference/inference.csv"), schema=schema, environment='INFERENCIA')

tfdv.display_anomalies(serving_anomalies)

#### Guardar cambios

In [45]:
tfdv.write_schema_text(schema, schema_path)

# Generación de nuevas estadísticas usando el esquema actualizado

In [46]:
user_schema_importer = Importer(
    source_uri=schema_gen.outputs['schema'].get()[0].uri, # directory containing only schema text proto
    artifact_type=standard_artifacts.Schema,
    reimport= True).with_id('schema_importer')

context.run(user_schema_importer)

compute_eval_stats = StatisticsGen(
      examples=example_gen.outputs['examples'],
      schema=user_schema_importer.outputs['result']
      )

context.run(compute_eval_stats)

ExecutionResult(
    component_id: StatisticsGen
    execution_id: 5
    outputs:
        statistics: OutputChannel(artifact_type=ExampleStatistics, producer_component_id=StatisticsGen, output_key=statistics, additional_properties={}, additional_custom_properties={}))

In [47]:
context.show(compute_eval_stats.outputs['statistics'])

## Comprobación de anomalías

In [48]:
validate_stats = ExampleValidator(
      statistics=compute_eval_stats.outputs['statistics'],
      schema=schema_gen.outputs['schema']
      )

In [49]:
context.run(validate_stats)

ExecutionResult(
    component_id: ExampleValidator
    execution_id: 6
    outputs:
        anomalies: OutputChannel(artifact_type=ExampleAnomalies, producer_component_id=ExampleValidator, output_key=anomalies, additional_properties={}, additional_custom_properties={}))

In [50]:
context.show(validate_stats.outputs['anomalies'])

,Anomaly short description,Anomaly long description
Feature name,,
'Cover_Type',The domain does not match the type,"The domain ""string_domain"" does not match the type: INT"


,Anomaly short description,Anomaly long description
Feature name,,
'Cover_Type',The domain does not match the type,"The domain ""string_domain"" does not match the type: INT"


# Ingeniería de características

In [51]:
def preprocessing_fn(inputs):
    
    outputs = {}
    
    # Transformación de features categóricos
    for categorical_feature in ['Wilderness_Area', 'Soil_Type']:
        outputs[categorical_feature] = tft.compute_and_apply_vocabulary(inputs[categorical_feature])
    
    # Transformación de features numéricos
    for numeric_feature in ['Elevation', 'Slope', 'Horizontal_Distance_To_Hydrology', 'Vertical_Distance_To_Hydrology', 'Horizontal_Distance_To_Roadways', 'Hillshade_9am', 'Hillshade_Noon', 'Horizontal_Distance_To_Fire_Points']:
        outputs[numeric_feature] = tft.scale_to_0_1(inputs[numeric_feature])
        
    outputs['Cover_Type'] = inputs['Cover_Type']
    
    return outputs

In [52]:
transform = Transform(
    examples=example_gen.outputs['examples'],
    schema=schema_gen.outputs['schema'],
    preprocessing_fn= "prep.preprocessing_fn"
)

In [53]:
context.run(transform)

2024-03-04 03:27:57.061554: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcuda.so.1'; dlerror: libcuda.so.1: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: /usr/local/cuda/lib64:/usr/local/cuda/lib:/usr/local/lib/x86_64-linux-gnu:/usr/local/nvidia/lib:/usr/local/nvidia/lib64:/usr/local/nvidia/lib:/usr/local/nvidia/lib64
2024-03-04 03:27:57.061590: W tensorflow/compiler/xla/stream_executor/cuda/cuda_driver.cc:265] failed call to cuInit: UNKNOWN ERROR (303)
2024-03-04 03:27:57.061634: I tensorflow/compiler/xla/stream_executor/cuda/cuda_diagnostics.cc:156] kernel driver does not appear to be running on this host (a2174d73310a): /proc/driver/nvidia/version does not exist
2024-03-04 03:27:57.061879: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical o

Instructions for updating:
Use ref() instead.


Instructions for updating:
Use ref() instead.


INFO:tensorflow:Assets written to: meta/Transform/transform_graph/7/.temp_path/tftransform_tmp/277bd443018e47729cee921ddad14794/assets


INFO:tensorflow:Assets written to: meta/Transform/transform_graph/7/.temp_path/tftransform_tmp/277bd443018e47729cee921ddad14794/assets


...

INFO:tensorflow:tensorflow_decision_forests is not available.


INFO:tensorflow:tensorflow_decision_forests is not available.


INFO:tensorflow:tensorflow_text is not available.


INFO:tensorflow:tensorflow_text is not available.


ExecutionResult(
    component_id: Transform
    execution_id: 7
    outputs:
        transform_graph: OutputChannel(artifact_type=TransformGraph, producer_component_id=Transform, output_key=transform_graph, additional_properties={}, additional_custom_properties={})
        transformed_examples: OutputChannel(artifact_type=Examples, producer_component_id=Transform, output_key=transformed_examples, additional_properties={}, additional_custom_properties={})
        updated_analyzer_cache: OutputChannel(artifact_type=TransformCache, producer_component_id=Transform, output_key=updated_analyzer_cache, additional_properties={}, additional_custom_properties={})
        pre_transform_schema: OutputChannel(artifact_type=Schema, producer_component_id=Transform, output_key=pre_transform_schema, additional_properties={}, additional_custom_properties={})
        pre_transform_stats: OutputChannel(artifact_type=ExampleStatistics, producer_component_id=Transform, output_key=pre_transform_stats, additional_properties={}, additional_custom_properties={})
        post_transform_schema: OutputChannel(artifact_type=Schema, producer_component_id=Transform, output_key=post_transform_schema, additional_properties={}, additional_custom_properties={})
        post_transform_stats: OutputChannel(artifact_type=ExampleStatistics, producer_component_id=Transform, output_key=post_transform_stats, additional_properties={}, additional_custom_properties={})
        post_transform_anomalies: OutputChannel(artifact_type=ExampleAnomalies, producer_component_id=Transform, output_key=post_transform_anomalies, additional_properties={}, additional_custom_properties={}))

In [54]:
transformed_statistics_gen = StatisticsGen(
    examples=transform.outputs['transformed_examples']
)

context.run(transformed_statistics_gen)

ExecutionResult(
    component_id: StatisticsGen
    execution_id: 8
    outputs:
        statistics: OutputChannel(artifact_type=ExampleStatistics, producer_component_id=StatisticsGen, output_key=statistics, additional_properties={}, additional_custom_properties={}))

In [55]:
context.show(transformed_statistics_gen.outputs['statistics'])

# Metadatos de aprendizaje automático

In [56]:
from ml_metadata.metadata_store import metadata_store
from ml_metadata.proto import metadata_store_pb2

In [57]:
# Instantiate a connection config
connection_config = metadata_store_pb2.ConnectionConfig()

connection_config = metadata_store_pb2.ConnectionConfig()
connection_config.sqlite.filename_uri = 'meta/metadata.sqlite'
connection_config.sqlite.connection_mode = 1  # READWRITE_OPENCREATE mode

store = metadata_store.MetadataStore(connection_config)

## Acceso a artefactos almacenados

### Tipos de artefactos almacenados

In [58]:
store.get_artifact_types()

[id: 14
 name: "Examples"
 properties {
   key: "span"
   value: INT
 }
 properties {
   key: "split_names"
   value: STRING
 }
 properties {
   key: "version"
   value: INT
 }
 base_type: DATASET,
 id: 16
 name: "ExampleStatistics"
 properties {
   key: "span"
   value: INT
 }
 properties {
   key: "split_names"
   value: STRING
 }
 base_type: STATISTICS,
 id: 18
 name: "Schema",
 id: 21
 name: "ExampleAnomalies"
 properties {
   key: "span"
   value: INT
 }
 properties {
   key: "split_names"
   value: STRING
 },
 id: 23
 name: "TransformGraph",
 id: 24
 name: "TransformCache"]

### Artefactos de tipo Esquema

In [59]:
store.get_artifacts_by_type("Schema")

[id: 3
 type_id: 18
 uri: "meta/SchemaGen/schema/3"
 custom_properties {
   key: "name"
   value {
     string_value: "schema:2024-03-04T03:27:19.947256"
   }
 }
 custom_properties {
   key: "producer_component"
   value {
     string_value: "SchemaGen"
   }
 }
 custom_properties {
   key: "state"
   value {
     string_value: "published"
   }
 }
 custom_properties {
   key: "tfx_version"
   value {
     string_value: "1.12.0"
   }
 }
 state: LIVE
 name: "schema:2024-03-04T03:27:19.947256"
 create_time_since_epoch: 1709522840704
 last_update_time_since_epoch: 1709522840794,
 id: 4
 type_id: 18
 uri: "meta/SchemaGen/schema/3"
 custom_properties {
   key: "is_external"
   value {
     int_value: 1
   }
 }
 custom_properties {
   key: "tfx_version"
   value {
     string_value: "1.12.0"
   }
 }
 create_time_since_epoch: 1709522864225
 last_update_time_since_epoch: 1709522864425,
 id: 10
 type_id: 18
 uri: "meta/Transform/pre_transform_schema/7"
 custom_properties {
   key: "name"
   value

## Seguimiento de artefactos

### Función

In [60]:
def obtener_artefactos_principales(store, output_artifact_id):
    """
    Devuelve los artefactos usados para generar el artefacto de salida especificado.
    
    Args:
    store: Un store Metadata conectada a ML Metadata.
    output_artifact_id: El ID del artefacto de salida.
    
    Returns:
    lista de artefactos de entrada usados para generar el artefacto de salida especificado.
    """
    input_artifacts = []

    # Encuentra todos los eventos asociados con el artefacto de salida
    events = store.get_events_by_artifact_ids([output_artifact_id])

    for event in events:
        if event.type == metadata_store_pb2.Event.OUTPUT:  # Busca eventos donde el artefacto es de salida
            # Encuentra la ejecución asociada con este evento
            execution_id = event.execution_id
            # Encuentra todos los eventos asociados con la ejecución para identificar artefactos de entrada
            related_events = store.get_events_by_execution_ids([execution_id])

            for rel_event in related_events:
                if rel_event.type == metadata_store_pb2.Event.INPUT:  # Busca eventos donde los artefactos son de entrada
                    artifact_id = rel_event.artifact_id
                    # Recupera el artefacto de entrada y lo añade a la lista
                    artifact = store.get_artifacts_by_id([artifact_id])[0]
                    input_artifacts.append(artifact)

    return input_artifacts

### Prueba

#### Examples

In [61]:
obtener_artefactos_principales(store, 2)[0].uri

'meta/CsvExampleGen/examples/1'

#### Schema

In [74]:
obtener_artefactos_principales(store, 5)[1].uri

'meta/SchemaGen/schema/3'